# Customer Segmentation Analysis via RFM+K-Means Clustering

### Purpose of running this analysis is to analyse the behaviour of customers and make strategies accordingly in order to drive more profits.
###  1. What's RFM
###  2. Create RFM Table
###  3. What's Kmeans clustering
###  4. Implement Kmeans clustering
###  5. Follow-up

## 1.1 What's RFM (customer value)?
### From WIKIPEDIA:
RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

#### RFM stands for the three dimensions:

* Recency – How recently did the customer purchase?-the number of days that have passed since the customer last purchased
* Frequency – How often do they purchase?-the number of purchases by the customer in the last 2.5 years 
* Monetary Value – How much do they spend?- in the last 2.5 years



#### RFM analysis readily answers these questions for our business…
* Who are my best customers?
* Which customers are at the verge of churning?
* Who has the potential to be converted in more profitable customers?
* Who are lost customers that you don’t need to pay much attention to?
* Which customers you must retain?
* Who are your loyal customers?
* Which group of customers is most likely to respond to your current campaign?
(Reference from Putler).

### Calculate RFM Values:
#### We need a few details of each Customers:

* Customer ID / Name / Company etc — to identify them
* Order date to calculate Recency (R) as days since last purchase: How many days ago was their last purchase? Deduct most recent purchase date from today to calculate the recency value. 1 day ago? 14 days ago? 500 days ago?
* Order id to calculate Frequency (F) as total number of transactions: How many times has the customer purchased from our store? For example, if someone placed 10 orders over a period of time, their frequency is 10.
* Order value to calculate Monetary (M) as total money spent: How many $$ (or whatever is your currency of calculation) has this customer spent? Simply total up the money from all transactions to get the M value.

In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import pandas_profiling as pp
from sklearn import preprocessing
from sklearn import cluster, tree, decomposition
from sklearn import metrics
from scipy.spatial.distance import cdist

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Import data 

In [ ]:
# import files with pandas
path = '/Users/.../data file'
all_files = glob.glob(path + '/*.csv', recursive=True)
dff = (pd.read_csv(f) for f in all_files)
df_up = pd.concat(dff)

In [ ]:
print('First 5 rows of dataframe')
df_up.head()

In [ ]:
print('#Row: {}'.format(df_up.shape[0]))
print('#columns:{}'.format(df_up.shape[1]))

In [ ]:
# date formatting
df_up['order_date']=pd.to_datetime(df_up['order_date'])
df_up['first_order_date']=pd.to_datetime(df_up['first_order_date'])
df_up['registered_date']=pd.to_datetime(df_up['registered_date'])
df_up['order_year']=df_up['order_date'].dt.year
df_up.head()

In [ ]:
df3years=df_up.loc[(df_up['order_year']==2016)| (df_up['order_year']==2017)|(df_up['order_year']==2018)]
df3years.head(5)

### Select variables for RMF analysis

In [ ]:
# create a new data frame to generate RFM table
df3years_sub=df3years[['customer_id','job_order_id','order_date','total_amount','qty']]
df3years_sub.head()

In [ ]:
print('Oldest order date', df3years_sub['order_date'].min())
print('Latest order date',df3years_sub['order_date'].max())

In [ ]:
# Set up end date for RFM analysis
import datetime as dt
End = dt.datetime(2018, 6, 10)

### 2.1 Generate RFM Table

In [ ]:
rfmTable = df3years_sub.groupby('customer_id').agg({'order_date': lambda x: (End - x.max()).days, 
                                                    'job_order_id': lambda x: len(x), 
                                                    'total_amount': lambda x: x.sum()}).reset_index()
rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'job_order_id': 'frequency', 
                         'total_amount': 'monetary_value'}, inplace=True)
print('First 5 rows of RFM Table')
rfmTable.head()

In [ ]:
corr = rfmTable[['recency','frequency','monetary_value']].corr()
sns.heatmap(corr)

#### From the correlation table, we can tell there is high correlation between frequency and monetary_value.


### 2.2 Check Correlation between RFM features

## 3.1 What's KMeans clustering?
### K-means clustering is a type of unsupervised learning, which is used when you have unlabeled data (i.e., data without defined categories or groups). The goal of this algorithm is to find groups in the data, with the number of groups represented by the variable K.-DataScience.com

## 3.2 How does KMeans work?
### The algorithm works iteratively to assign each data point to one of K groups based on the features that are provided. Data points are clustered based on feature similarity.
#### Steps:
* Step 1: Initialisation – K initial centroids (cluster center) are generated randomly
* Step 2: Assignment – Assign observations to the nearest cluster center via finding the min(distance between the points and the clusters. (arg min D(Pi,Kn), different definition of distance).
* Step 3: Move the centroids to their new position– For each cluster that we got in Step2, we recompute its centorid position by averaging all features of all data points within that cluster. (New Centroid Position= sum(vectors(Pi))/sum(i) for each cluster).
* Step 4: Repeat Step 3 & Step 4, iteratively until convergence (centroids don't more any more)

## 3.1 Use KMeans Clustering to run customer segamentation

In [ ]:
# Data preprocessing
data=rfmTable.copy()
var_list=['recency','frequency','monetary_value']
for i in var_list:
    data[i]=preprocessing.scale(data[i])

### How many clusters? - determine K
A good clustering is one that achieves:
High within-cluster similarity
Low inter-cluster similarity

We will use elbow method to find the optimal K. The idea of the elbow method is to run k-means clustering on the dataset for a range of values of k (say, k from 1 to 10 in the example below), and for each value of k calculate the sum of squared errors (SSE).The idea is that we want a small SSE, but that the SSE tends to decrease toward 0 as we increase k (the SSE is 0 when k is equal to the number of data points in the dataset, because then each data point is its own cluster, and there is no error between it and the center of its cluster). So our goal is to choose a small value of k that still has a low SSE, and the elbow usually represents where we start to have diminishing returns by increasing k.

WCSSE= SUM(distance(Pi, K1)^2) + SUM(distance(Pi, K2)^2) + SUM(distance(Pi, K3)^2)+... (each Pi within that K cluster)

In [ ]:
#Finding the optimum number of clusters for k-means classification
WCSSE = []
K = range(1,11)
for k in K:
    kmeanModel = cluster.KMeans(n_clusters=k,random_state=123)
    kmeanModel.fit(data[var_list])
    WCSSE.append(kmeanModel.inertia_) #value of the inertia criterion:sum of squared distances to the closest centroid for all observations in the training set

cluster_df=pd.DataFrame({'num_clusters':K, 'SSE within Cluster':WCSSE})
print(cluster_df)
# Plot the elbow
plt.plot(K, WCSSE, marker='b*-')
plt.xlabel('Numbers of clusters (k)')
plt.ylabel('Sum of Squared Error (Within Cluster)')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# @hidden_cell
'But what happened if the sample sizes are very different across all clusters? We need to normalize our data when calculate WCSSE'

In [ ]:
#Finding the optimum number of clusters for k-means classification
from sklearn import preprocessing
from sklearn import cluster, tree, decomposition
from sklearn import metrics
from scipy.spatial.distance import cdist

distortions = []
K = range(1,11)
for k in K:
    kmeanModel = cluster.KMeans(n_clusters=k,).fit(data[var_list])
    kmeanModel.fit(data[var_list])
    distortions.append(sum(np.min(cdist(data[var_list], kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / data[var_list].shape[0])
     #normalization term / data.shape[0]
# Plot the elbow
plt.plot(K, distortions, marker='b*-')
plt.xlabel('Numbers of clusters (k)')
plt.ylabel('Distortions')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Number of clusters
kmeans = KMeans(n_clusters=3)
# Fitting the input data
kmeans = kmeans.fit(X)
# Getting the cluster labels
labels = kmeans.predict(X)
# Centroid values
centroids = kmeans.cluster_centers_

# Comparing with scikit-learn centroids
print(C) # From Scratch
print(centroids) # From sci-kit learn

### From the plot above, we got our optimal k=6

In [ ]:
# k=6
km6 = cluster.KMeans(n_clusters=6,random_state=123)
data2=data.copy()
data2['cluster'] = km6.fit_predict(data2[var_list])
print('First 10 rows')
data2.head(10)

In [ ]:
km6.fit(data2)

### How many customers are assigned into each cluster?

In [ ]:
df_k6 = pd.merge(rfmTable, data2[['customer_id','cluster']], on='customer_id')
df_k6.groupby('cluster')['customer_id'].count().reset_index(name='# of customers')

#### KMeans finds outliers nicely and groups them into one cluster~ will check by visulization next~

### Use boxplot to visualize each cluster 

In [ ]:
sns.boxplot(x="cluster", y="frequency", data=df_k6).set_title('Order Counts of each cluster')  # RUN PLOT   
plt.figure(1, figsize=(9, 6))
plt.show()

In [ ]:
sns.boxplot(x="cluster", y="monetary_value", data=df_k6).set_title('Total Sales of each cluster') # RUN PLOT   
plt.figure(1, figsize=(9, 6))
plt.show()

### Look at mean value within each clusters

In [ ]:
df_k6[var_list+['cluster']].groupby('cluster').mean().reset_index()

### Look at median value within each clusters

In [ ]:
df_k6[var_list+['cluster']].groupby('cluster').median().reset_index()

In [ ]:
# Put them together
grouped=df_k6.groupby(['cluster']).agg({ 'cluster' : ['count'],
        'recency': ['mean', 'median', 'min'] ,
        'frequency': ['mean', 'median', 'min'],
       'monetary_value': ['mean', 'median', 'min']}).reset_index()
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()] 
grouped.sort_values('cluster_count', ascending=False)

### Cluster =4 & 5 are selected due to its high stat of Frequency (Transcation) and Monetary_value (Total Sales) and generate a high value customer list for Facebook LookAlike campaign.

### 5. Follow-up
* Use other clustering method to find optimial K when elbow method doesn't work
* Test different customer groups (cluster=0, 1 and 3) via retargeting campaign for repeat customers.
* Retention team needs to review KPIs and has a working session with analytics team in order to deploy marketing strategy and plan. 
### 6. Other analysis for retention team
* Shopping basket analysis based on per order details. Figure out which products that customer would like to purchase together.

### Check statistic within all groups

In [ ]:
pp.ProfileReport(df_k6)